# The netCDF file format

The [netCDF](https://www.unidata.ucar.edu/software/netcdf/) file format is a popular scientific file format for ocean and atmosphere gridded datasets. It is a collection of formats storing arrays:

* netCDF classic
    * more widespread
    * 2 GB file limit (if you don't use the unlimited dimension)
    * often preferred for distributing products

* netCDF 64 bit offset
    * supports larger files

* NetCDF4
    * based on [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format)
    * allows compression
    * multiple unlimited variables

netCDF was developed by Unidata-UCAR with the aim of storing climate model data (3D + time). netCDF format allows auxilary information about each variable to be added. It can have a readable text equivalent (e.g. using [ncdump](http://www.bic.mni.mcgill.ca/users/sean/Docs/netcdf/guide.txn_79.html#:~:text=The%20ncdump%20tool%20generates%20an,variable%20data%20in%20the%20file.&text=Thus%20ncdump%20and%20ncgen%20can,between%20binary%20and%20ASCII%20representations.)) and can be used with [Climate and Forecast (CF)](http://cfconventions.org/) data convention.

## Data model

|                 |                                                              |
| --------------- | -------------------------------------------------------------|
| **Dimensions**  | describe the axes of the data arrays                         |
| **Variables**   | N-dimensional arrays of data                                 |
| **Attributes**  | small note/supplementary metadata as annotations to the file |


Ocean model dataset example:
 
|   |   |   |   |   |
|---|---|---|---|---|
| Dimensions | lat | lon | depth | time |
| Variable | Temperature | Salinity |  |  |
| Global attribute | Geographic grid type | History |  |  |
| Variable attributes | Long_name: "sea water temperature" | Missing_value: 1.09009E36 | Units: deg. C | range: -2:50 |

<br>
<img src="../figures/dataset-diagram.png" style="width:600px";/>

## Python packages

The main Python interface to the netCDF C library is [netCDF4](http://unidata.github.io/netcdf4-python/) package. In this tutorial, however, we will use a more high-level package called [xarray](http://xarray.pydata.org/en/stable/index.html).

Note: another good package for netCDF files is [iris](http://scitools.org.uk/iris/) (developed by the UK Met Office).

## Working with netCDF files using xarray

[xarray](http://xarray.pydata.org/) package brings the power of pandas to environmental sciences by providing N-dimensional variants of the core pandas data structures.

| Pandas | xarray  |
|---|---|
| Series  | DataArray  |
| DataFrame  | Dataset  |


DataArray uses names of dimensions making it easier to track than by using axis numbers.

Compare:
```python
# xarray style
>>> ds.sel(time='2018-01-12').max(dim='ensemble')

# numpy style
>>> array[[0, 1, 2, 3], :, :].max(axis=2)
```

(Taken from Stephan Hoyer's [ECMWF talk](https://docs.google.com/presentation/d/16CMY3g_OYr6fQplUZIDqVtG-SKZqsG8Ckwoj2oOqepU/edit#slide=id.g2b68f9254d_1_27))

**Main advantages of using xarray versus plain netCDF4:**
* intelligent selection along labelled dimensions (and also indexes)
* groupby operations
* data alignment
* IO (netcdf)
* conversion from and to pandas `DataFrame` objects

In [1]:
# Import everything that we are going to need
import pandas as pd
import xarray as xr
import numpy as np

## Import a local dataset

In [2]:
GETM = xr.open_dataset('../data/cefas_GETM_nwes.nc4')
GETM

<xarray.Dataset>
Dimensions:     (latc: 360, level: 5, lonc: 396, time: 6)
Coordinates:
  * latc        (latc) float64 45.4 45.45 45.5 45.55 ... 63.2 63.25 63.3 63.35
  * level       (level) float64 1.0 6.0 11.0 16.0 21.0
  * lonc        (lonc) float64 -17.5 -17.42 -17.34 -17.26 ... 13.94 14.02 14.1
  * time        (time) datetime64[ns] 1996-02-01T01:00:00 ... 1996-02-06T01:0...
Data variables:
    bathymetry  (latc, lonc) float64 ...
    h           (time, level, latc, lonc) float32 ...
    temp        (time, level, latc, lonc) float32 ...
Attributes:
    title:                     North West European Shelf 3nm
    history:                   Sun Nov 20 00:54:48 2016: ncks -v time,latc,lo...
    NCO:                       4.4.8
    nco_openmp_thread_number:  1

We can see that this file holds bathymetry, height and tempearture data for a gridded area of the North West European Shelf.

In [3]:
GETM.dims

Frozen(SortedKeysDict({'latc': 360, 'lonc': 396, 'time': 6, 'level': 5}))

We can print these dimensions to see the extent of the netCDF

In [5]:
GETM.latc
#GETM.lonc
#GETM.time
#GETM.level

<xarray.DataArray 'latc' (latc: 360)>
array([45.400002, 45.450002, 45.500002, ..., 63.249999, 63.299999, 63.349999])
Coordinates:
  * latc     (latc) float64 45.4 45.45 45.5 45.55 45.6 ... 63.2 63.25 63.3 63.35
Attributes:
    units:    degrees_north

In [6]:
print(type(GETM.coords['latc']))
GETM.coords['latc'].shape

<class 'xarray.core.dataarray.DataArray'>


(360,)

In [7]:
# List name of dataset attributes
GETM.attrs.keys()

dict_keys(['title', 'history', 'NCO', 'nco_openmp_thread_number'])

In [8]:
# List variable names
GETM.data_vars.keys()

KeysView(Data variables:
    bathymetry  (latc, lonc) float64 ...
    h           (time, level, latc, lonc) float32 ...
    temp        (time, level, latc, lonc) float32 ...)

This tells  us that bathymetry is 2D, varying only with latitude and longitude. Height and temperature are 4D, varying with time and model level as well.

### Extract variable from dataset

In [9]:
temp = GETM['temp']
print(type( temp ))
temp.shape

<class 'xarray.core.dataarray.DataArray'>


(6, 5, 360, 396)

### Access variable attributes

In [10]:
# print variable attributes

for at, val in temp.attrs.items():
    print(f'{at:<15}: {val}')

units          : degC
long_name      : temperature
valid_range    : [-2. 40.]


## Accessing data values

In [11]:
temp[0, 0, 90, 100]

<xarray.DataArray 'temp' ()>
array(11.532867)
Coordinates:
    latc     float64 49.9
    level    float64 1.0
    lonc     float64 -9.5
    time     datetime64[ns] 1996-02-01T01:00:00
Attributes:
    units:        degC
    long_name:    temperature
    valid_range:  [-2. 40.]

In [12]:
temp.values[0, 0, 90, 100]

11.532867

## Indexing and selecting data

Xarray indexing overview can be found [here](http://xarray.pydata.org/en/stable/getting-started-guide/quick-overview.html#indexing). The graphics below is a summary what one can do:

<br>
<img src="../figures/xarray_indexing_table.png" style="width:600px";/>

In [ ]:
# positional by integer
print(temp[0, 2, :, :].shape)

# positional by label
print(temp.loc['1996-02-02T01:00:00', :, :, :].shape)

# by name and integer
print(temp.isel(level=1, latc=90, lonc=100).shape)

# by name and label
print(temp.sel(time='1996-02-02T01:00:00').shape)
# temp.loc

### Define selection using nearest value

In [ ]:
# GETM.sel(level=1)['temp']
GETM['temp'].sel(level=2, lonc=-5.0, latc=50.0, method='nearest')

In [ ]:
# Use tolerance for label selection (lat=-50 should not yield data)
tol = 0.5

try:
    GETM['temp'].sel(level=1, lonc=-5.0, latc=-50.0, method='nearest', tolerance=tol)
except KeyError:
    print(f'ERROR: outside tolerance of {tol}')

## Plotting

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs

Define a general mapping function using cartopy

In [ ]:
def make_map():
    # create figure and axes instances
    fig = plt.figure()
    ax = fig.add_subplot(111, projection=ccrs.Stereographic(central_latitude=60))
    #ax.coastlines(resolution='50m', linewidth=0.5)
    ax.set_extent([-10, 15, 49, 60], crs=ccrs.PlateCarree())
    
    gl = ax.gridlines(draw_labels=False)
    
    feature = cartopy.feature.NaturalEarthFeature(name='coastline',
                                                  category='physical',
                                                  scale='50m',
                                                  edgecolor='0.5',
                                                  facecolor='0.8')
    ax.add_feature(feature)
    return fig, ax

make_map()
plt.show()

In [ ]:
latc = GETM.coords['latc']
lonc = GETM.coords['lonc']

var = GETM.temp.sel(time='1996-02-02T01:00:00', level=21)

# create arrays of coordinates for contourf
# lon2d, lat2d = np.meshgrid(lonc, latc)

fig, ax = make_map()
# draw filled contours.
h = ax.contourf(lonc, latc, var, 50, cmap=plt.cm.coolwarm, transform=ccrs.PlateCarree())

# add colorbar.
cbar = fig.colorbar(h)
cbar.set_label(var.units)

# add title
ax.set_title(f'A slice of {var.long_name}');

In [ ]:
# But often, this will do
# GETM.temp.isel(time=0, level=0).plot();
GETM.temp.sel(time='1996-02-02T01:00:00', level=21).plot()
plt.show()

## Arithmetic operations

In [ ]:
top = GETM['temp'].isel(time=0, level=4)
bottom = GETM['temp'].isel(time=0, level=0)

diff = top - bottom

diff.plot()
plt.show()

### Calculate average along a dimension

In [ ]:
# average over time
time_ave = GETM['temp'].mean('time')

# average over time and level (vertical)
time_and_level_ave = GETM['temp'].mean(['time','level'])

time_and_level_ave.plot()
plt.show()

In [ ]:
# zonal average (vertical)
timelon_ave = GETM['temp'].mean(['time','lonc']).isel(level=4)

timelon_ave.plot()
plt.show()

Using **xarray** we have the data access power of **netCDF4** with all the intelligent selection, arithmetic, statistical methods and plotting of **pandas**.

## A dataset can easily be saved to a netCDF file

In [ ]:
ds = GETM[['temp']].mean('time','level')
ds.to_netcdf('../data/temp_avg_level_time.nc')

## Exercise 1

* Extract the bathymetry
* Extract the time averaged seabed temperature at level = 0
* Produce a scatter plot of depth vs. seabed temperature

In [ ]:
GETM.data_vars.keys()

In [ ]:
# Your code here

## Import remote dataset

xarray supports [OpenDAP](https://www.opendap.org/). This means that a dataset can be accessed remotely and subsetted as needed. Only the selected parts are downloaded.

In [ ]:
remote_data = xr.open_dataset(
      'http://iridl.ldeo.columbia.edu/SOURCES/.OSU/.PRISM/.monthly/dods',
      decode_times=False)
remote_data

## Excercise 2
Import data from a netCDF or csv file and start exploring it. Some ideas:
- Use pandas to get quick statistics
- Do some data cleaning and calculations with numpy
- Plot it up with matplotlib, pandas, seaborn or cartopy as you prefer

I encourage you to use your own data if you have some. If not, we have some sample datasets you can explore.

### Earthquake data (csv file)
US Geological Survey (USGS) provides various [earthquakes data](https://earthquake.usgs.gov/data/data.php#eq) on a global scale. Its Earthquake Catalog contains earthquake source parameters (e.g. hypocenters, magnitudes, phase picks and amplitudes) and other products (e.g. moment tensor solutions, macroseismic information, tectonic summaries, maps) produced by contributing seismic networks.

If you follow this [link](http://earthquake.usgs.gov/earthquakes/search/), you can search throught the catalog and filter data by the magnitude, time and geographic region. In the `data/` folder, we provide an [example dataset](../data/earthquakes_2015_2016_gt45.csv) of earthquakes with magnitude >4.5 that occurred around the world over the period of a year.

To get you started, the following cell loads the data into a pandas DataFrame

In [ ]:
# import pandas as pd
# df = pd.read_csv('../data/earthquakes_2015_2016_gt45.csv', parse_dates = ['time',], index_col='time')
# df.head()

If you want to build your project on these data, some possible ideas are:
* `pandas` package will be most useful to read in the data, as well as analyse them
* Use `cartopy` to plot the data using longitude and latitude columns
* Explore `pandas`' `groupby()` method, which you can use to aggregate data by time or other parameter
* Create a histogram of earthquakes magnitude

### Arctic Sea Ice (netCDF files)
* In this project you are offered to use NOAA/NSIDC Climate Data Record of Passive Microwave Sea Ice Concentration.
* In the `../data/` directory, there are 2 netCDF files `seaice_conc_monthly*` that correspond to September 1991  and September 2012 .
* If you want to download data for other months, visit the [NSIDC's data portal](https://nsidc.org/data/search/#keywords=sea+ice/sortKeys=score,,desc/facetFilters=%257B%257D/pageNumber=1/itemsPerPage=25).

For this project, I recommend that you:
* use `xarray` for opening and reading the netCDF files
* use `cartopy` for creating a plot with a correct map projection
* use appropriate colormaps for the sea ice concentration and difference between the two years

Some code to get you started:

In [ ]:
# import cartopy.crs as ccrs
# import matplotlib.pyplot as plt
# import xarray as xr

In [ ]:
# ds1 = xr.open_dataset('../data/seaice_conc_monthly_nh_f08_199109_v02r00.nc')
# ds2 = xr.open_dataset('../data/seaice_conc_monthly_nh_f17_201209_v02r00.nc')

## References

* https://www.unidata.ucar.edu/software/netcdf/docs/netcdf_data_model.html
* http://xarray.pydata.org/en/stable/getting-started-guide/quick-overview.html#indexing